# Greenlet

如果我们在单个线程内有20个任务，要想实现在多个任务之间切换，

使用yield生成器的方式过于麻烦（需要先得到初始化一次的生成器，然后再调用send。。。非常麻烦），

而使用greenlet模块可以非常简单地实现这20个任务直接的切换

In [1]:
from greenlet import greenlet

def eat(name):
    print(f'{name} eat 1')
    g2.switch('egon')
    print(f'{name} eat 2')
    g2.switch()
    
def play(name):
    print(f'{name} play 1')
    g1.switch()
    print(f'{name} play 2')

g1=greenlet(eat)
g2=greenlet(play)

g1.switch('egon')#可以在第一次switch时传入参数，以后都不需要

egon eat 1
egon play 1
egon eat 2
egon play 2


单纯的切换（在没有io的情况下或者没有重复开辟内存空间的操作），反而会降低程序的执行速度

In [2]:
#顺序执行
import time
def f1():
    res=1
    for i in range(100000000):
        res+=i

def f2():
    res=1
    for i in range(100000000):
        res*=i

start=time.time()
f1()
f2()
stop=time.time()
print(f'run time is {stop - start}')



run time is 9.597684383392334


In [3]:
#切换
from greenlet import greenlet
import time
def f1():
    res=1
    for i in range(100000000):
        res+=i
        g2.switch()

def f2():
    res=1
    for i in range(100000000):
        res*=i
        g1.switch()

start=time.time()
g1=greenlet(f1)
g2=greenlet(f2)
g1.switch()
stop=time.time()
print(f'run time is {stop - start}')

run time is 55.699546337127686


greenlet只是提供了一种比generator更加便捷的切换方式，当切到一个任务执行时如果遇到io，那就原地阻塞，仍然是没有解决遇到IO自动切换来提升效率的问题。

单线程里的这20个任务的代码通常会既有计算操作又有阻塞操作，我们完全可以在执行任务1时遇到阻塞，就利用阻塞的时间去执行任务2。。。。如此，才能提高效率，这就用到了Gevent模块。